# Introduction aux descripteurs

Dans les deux chapitres précédents de notre tutoriel Python, nous avons appris à utiliser les propriétés Python et même à implémenter une classe de propriétés sur mesure. Dans ce chapitre, vous allez apprendre les détails des descripteurs.

Les descripteurs ont été introduits dans Python dans la version 2.2. À cette époque, le document "What's New in Python2.2" mentionnait : "La seule grande idée sous-jacente au nouveau modèle de classe est qu'une API pour décrire les attributs d'un objet à l'aide de descripteurs a été formalisée. Les descripteurs précisent la valeur d'un attribut, en indiquant s'il s'agit d'une méthode ou d'un champ. Avec l'API des descripteurs, les méthodes statiques et les méthodes de classe deviennent possibles, ainsi que des constructions plus exotiques."

Un descripteur est un attribut d'objet avec un __comportement de liaison__, un attribut dont l'accès a été remplacé par des méthodes dans le protocole des descripteurs. Ces méthodes sont ```__get__()```, ```__set__()``` et ```__delete__()```.

Si l'une de ces méthodes est définie pour un objet, on dit qu'il s'agit d'un descripteur.

Leur but consiste à fournir aux programmeurs la possibilité d'ajouter des attributs gérés aux classes. Les descripteurs sont introduits pour obtenir, définir ou supprimer des attributs du dictionnaire ```__dict__``` de l'objet via les méthodes mentionnées ci-dessus. L'accès à un attribut de classe lance la chaîne de recherche.

<center><img src="img/illustration4.png" width="50%"></center>

Examinons de plus près ce qui se passe. 

Supposons que nous ayons un objet ```obj``` : que se passe-t-il si nous essayons d'accéder à un attribut (ou une propriété) ap ? "Accéder" à l'attribut signifie "obtenir" la valeur, l'attribut est donc utilisé par exemple dans une fonction d'impression ou à l'intérieur d'une expression. L'objet et la classe appartenant au ```type(obj)``` contiennent tous deux un attribut de dictionnaire ```__dict__```. Cette situation est viuslaisée dans le schéma suivant :

<center><img src="img/illustration5.png" width="70%"></center>

```obj.ap``` a une chaîne de recherche commençant par ```obj.__dict__['ap']```, c'est-à-dire qu'il vérifie si```obj.ap``` est une clé du dictionnaire ```obj.__dict__['ap']```.

<center><img src="img/illustration6.png" width="70%"></center>

Si ap n'est pas une clé de ```obj.__dict__```, il essaiera de rechercher ```type(obj).__dict__['ap']```

<center><img src="img/illustration7.png" width="70%"></center>

Si obj n'est pas non plus contenu dans ce dictionnaire, il continuera à vérifier les classes de base du type(ap) en excluant les métaclasses.

Nous le démontrons dans un exemple :


In [1]:
class A:
    ca_A = "class attribute of A"
    
    def __init__(self):
        self.ia_A = "instance attribute of A instance"
        
        
class B(A):
    ca_B = "class attribute of B"
    
    def __init__(self):
        super().__init__()
        self.ia_B = "instance attribute of A instance"
        
        
x = B()
print(x.ia_B)
print(x.ca_B)
print(x.ia_A)
print(x.ca_A)

instance attribute of A instance
class attribute of B
instance attribute of A instance
class attribute of A


Si nous appelons ```print(x.non_existing)```, nous obtenons l'exception suivante :

In [2]:
print(x.non_existing)

AttributeError: 'B' object has no attribute 'non_existing'

Si la valeur recherchée est un objet définissant l'une des méthodes du descripteur, Python peut outrepasser le comportement par défaut et invoquer la méthode du descripteur à la place. L'endroit où cela se produit dans la chaîne de précédence dépend des méthodes de descripteurs qui ont été définies.

Les descripteurs fournissent un protocole puissant et général, qui est le mécanisme sous-jacent aux propriétés, méthodes, méthodes statiques, méthodes de classe et ```super()```. Les descripteurs étaient nécessaires pour mettre en œuvre les classes dites de nouveau style introduites dans la version 2.2. Les "nouvelles classes de style" sont aujourd'hui les classes par défaut.

## Protocole des descripteurs

Le protocole général des descripteurs est constitué de trois méthodes :

```python
descr.__get__(self, obj, type=None) -> valeur
descr.__set__(self, obj, valeur) -> None
descr.__delete__(self, obj) -> None
```

Si vous définissez une ou plusieurs de ces méthodes, vous allez créer un descripteur. Nous distinguons les descripteurs de données des descripteurs de non-données :

__descripteur de non-données__
Si nous définissons uniquement la méthode ```__get__()```, nous créons un descripteur non-data, qui est principalement utilisé pour les méthodes.

__descripteur de données__
Si un objet définit la méthode ```__set__()``` ou ```__delete__()```, il est considéré comme un descripteur de données. Pour créer un descripteur de données en lecture seule, définissez à la fois ```__get__()``` et ```__set__()```, la ```__set__()``` générant une AttributeError lorsqu'elle est appelée. Il suffit de définir la méthode ```__set__()``` avec un caractère générique levant une exception pour en faire un descripteur de données.

Nous en arrivons enfin à notre exemple de descripteur simple dans le code suivant :

In [3]:
class SimpleDescriptor(object):
    """
    A simple data descriptor that can set and return values
    """
    
    def __init__(self, initval=None):
        print("__init__ of SimpleDecorator called with initval: ", initval)
        self.__set__(self, initval)
        
    def __get__(self, instance, owner):
        print(instance, owner)
        print('Getting (Retrieving) self.val: ', self.val)
        return self.val
    
    def __set__(self, instance, value):
        print('Setting self.val to ', value)
        self.val = value
        
        
class MyClass(object):
    x = SimpleDescriptor("green")
    
    
m = MyClass()
print(m.x)
m.x = "yellow"
print(m.x)

__init__ of SimpleDecorator called with initval:  green
Setting self.val to  green
<__main__.MyClass object at 0x10d3db650> <class '__main__.MyClass'>
Getting (Retrieving) self.val:  green
green
Setting self.val to  yellow
<__main__.MyClass object at 0x10d3db650> <class '__main__.MyClass'>
Getting (Retrieving) self.val:  yellow
yellow


Le troisième paramètre owner de ```__get__``` est toujours la classe propriétaire et fournit aux utilisateurs une option pour faire quelque chose avec la classe qui a été utilisée pour appeler le descripteur. Habituellement, c'est-à-dire si le descripteur est appelé via un objet obj, le type de l'objet peut être déduit en appelant type(obj). La situation est différente, si le descripteur est invoqué à travers une classe. Dans ce cas, c'est None et il ne serait pas possible d'accéder à la classe à moins que le troisième argument ne soit donné. Le deuxième paramètre instance est l'instance par laquelle l'attribut a été accédé, ou None si l'attribut est accédé par le propriétaire.

Jetons un coup d'oeil aux dictionnaires ```__dict__``` des instances et de la classe :

```x``` est un attribut de classe dans la classe précédente. Vous vous êtes peut-être demandé si nous pouvions également utiliser ce mécanisme dans la méthode ```__init__``` pour définir un attribut d'instance. Cela n'est pas possible. Les méthodes ```__get__()```,```__set__()``` et ```__delete__()``` ne s'appliquent que si une instance de la classe contenant la méthode (une classe dite descriptive) apparaît dans une classe propriétaire (le descripteur doit se trouver soit dans le dictionnaire de classes du propriétaire, soit dans le dictionnaire de classes de l'un de ses parents). Dans les exemples ci-dessus, l'attribut ```x``` est dans le ```__dict__``` propriétaire de la classe propriétaire ```MyClass```, comme on peut le voir ci-dessous :

In [4]:
print(m.__dict__)
print(MyClass.__dict__)
print(SimpleDescriptor.__dict__)

{}
{'__module__': '__main__', 'x': <__main__.SimpleDescriptor object at 0x10d3d0210>, '__dict__': <attribute '__dict__' of 'MyClass' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass' objects>, '__doc__': None}
{'__module__': '__main__', '__doc__': '\n    A simple data descriptor that can set and return values\n    ', '__init__': <function SimpleDescriptor.__init__ at 0x10cfdd760>, '__get__': <function SimpleDescriptor.__get__ at 0x10cfdd940>, '__set__': <function SimpleDescriptor.__set__ at 0x10cfdd800>, '__dict__': <attribute '__dict__' of 'SimpleDescriptor' objects>, '__weakref__': <attribute '__weakref__' of 'SimpleDescriptor' objects>}


print(MyClass.__dict__['x'])

Il est possible d'appeler un descripteur directement par son nom de méthode, par exemple ```d.__get__(obj)```.

Alternativement, il est plus courant qu'un descripteur soit invoqué automatiquement lors de l'accès à un attribut. Par exemple, ```obj.d``` cherche ```d``` dans le dictionnaire ```__dict__``` de obj. Si d définit la méthode ```__get__()```, alors ```d.__get__(obj)``` est invoqué selon les règles de précédence énumérées ci-dessous.

Il y a une différence si ```obj``` est un objet ou une classe :

Pour les objets, la méthode permettant de contrôler l'invocation se trouve dans```object.__getattribute__()``` qui transforme ```b.x``` en l'appel ```type(b).__dict__['x'].__get__(b, type(b))```. L'implémentation fonctionne par le biais d'une chaîne de priorité qui donne aux descripteurs de données la priorité sur les variables d'instance, aux variables d'instance la priorité sur les descripteurs non-données, et attribue la priorité la plus basse à ```__getattr__()``` si elle est fournie.

Pour les classes, la méthode correspondante se trouve dans la classe de type, c'est-à-dire ```type.__getattribute__()``` qui transforme ```B.x``` en ```B.__dict__['x'].__get__(None, B)```.

```__getattribute__``` n'est pas implémenté en Python mais en C. Le code Python suivant est une simulation de la logique en Python. Nous pouvons voir que les descripteurs sont appelés par les implémentations de ```__getattribute__```.

In [6]:
def __getattribute__(self, key):
    "Emulate type_getattro() in Objects/typeobject.c"
    v = type.__getattribute__(self, key)
    if hasattr(v, '__get__'):
        return v.__get__(None, self)
    return v

m.__getattribute__("x")

<__main__.MyClass object at 0x10d3db650> <class '__main__.MyClass'>
Getting (Retrieving) self.val:  yellow


'yellow'

L'objet renvoyé par ```super()``` possède également une méthode ```__getattribute__()``` personnalisée pour invoquer les descripteurs. La recherche d'attributs ```super(B, obj).m``` recherche dans ```obj.__class__.__mro__``` la classe de base A qui suit immédiatement B et renvoie ensuite ```A.__dict__['m'].__get__(obj, B)```. Si ce n'est pas un descripteur, m est retourné inchangé. S'il n'est pas dans le dictionnaire, m revient à une recherche en utilisant ```object.__getattribute__()```.

Les détails ci-dessus montrent que le mécanisme pour les descripteurs est intégré dans les méthodes ```__getattribute__()``` pour object, type, et ```super()```. Les classes héritent de ce mécanisme lorsqu'elles dérivent d'object ou si elles ont une méta-classe fournissant une fonctionnalité similaire. Cela signifie également que l'on peut désactiver les appels automatiques aux descripteurs en surchargeant ```__getattribute__()```.

In [7]:
from weakref import WeakKeyDictionary

class Voter:
    required_age = 18 # in Germany
    
    def __init__(self):
        self.age = WeakKeyDictionary()
        
    def __get__(self, instance_obj, objtype):
        return self.age.get(instance_obj)
    
    def __set__(self, instance, new_age):
        if new_age < Voter.required_age:
            msg = '{name} is not old enough to vote in Germany'
            raise Exception(msg.format(name=instance.name))
        self.age[instance] = new_age
        print('{name} can vote in Germany'.format(
            name=instance.name))
        
    def __delete__(self, instance):
        del self.age[instance]
        
        
class Person:
    voter_age = Voter()
    def __init__(self, name, age):
        self.name = name
        self.voter_age = age
        
        
p1 = Person('Ben', 23)
p2 = Person('Emilia', 22)
p2.voter_age

Ben can vote in Germany
Emilia can vote in Germany


22

Un exemple pur d'implémentation d'une classe ```property()``` [doc python](https://docs.python.org/fr/3/howto/descriptor.html#id24)

In [8]:
class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"
    
    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc
        
    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)
    
    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)
        
    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)
        
    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)
    
    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)
    
    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

Une classe simple utilisant notre implémentation de la propriété.

In [9]:
class A:
    
    def __init__(self, prop):
        self.prop = prop
        
    @Property
    def prop(self):
        print("The Property 'prop' will be returned now:")
        return self.__prop
    
    @prop.setter
    def prop(self, prop):
        print("prop will be set")
        self.__prop = prop

Utilisation de notre classe A :

In [10]:
print("Initializing the Property 'prop' with the value 'Python'")
x = A("Python")
print("The value is: ", x.prop)
print("Reassigning the Property 'prop' to 'Python descriptors'")
x.prop = "Python descriptors"
print("The value is: ", x.prop)

Initializing the Property 'prop' with the value 'Python'
prop will be set
The Property 'prop' will be returned now:
The value is:  Python
Reassigning the Property 'prop' to 'Python descriptors'
prop will be set
The Property 'prop' will be returned now:
The value is:  Python descriptors


Rendons notre implémentation de propriété un peu plus bavarde avec quelques fonctions d'impression pour voir ce qui se passe :

In [11]:
class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"
    
    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        print("\n__init__ of Property called with:")
        print("fget=" + str(fget) + " fset=" + str(fset) + \
              " fdel=" + str(fdel) + " doc=" + str(doc))
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc
        
    def __get__(self, obj, objtype=None):
        print("\nProperty.__get__ has been called!")
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)
    
    def __set__(self, obj, value):
        print("\nProperty.__set__ has been called!")
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)
        
    def __delete__(self, obj):
        print("\nProperty.__delete__ has been called!")
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)
        
    def getter(self, fget):
        print("\nProperty.getter has been called!")
        return type(self)(fget, self.fset, self.fdel, self.__doc__)
    
    def setter(self, fset):
        print("\nProperty.setter has been called!")
        return type(self)(self.fget, fset, self.fdel, self.__doc__)
    
    def deleter(self, fdel):
        print("\nProperty.deleter has been called!")
        return type(self)(self.fget, self.fset, fdel, self.__doc__)
    
    
class A:
    
    def __init__(self, prop):
        self.prop = prop
        
    @Property
    def prop(self):
        """ This will be the doc string of the property """
        print("The Property 'prop' will be returned now:")
        return self.__prop
    
    @prop.setter
    def prop(self, prop):
        print("prop will be set")
        self.__prop = prop
        
    def prop2_getter(self):
        return self.__prop2
    
    def prop2_setter(self, prop2):
        self.__prop2 = prop2
        
    prop2 = Property(prop2_getter, prop2_setter)
    
    
print("Initializing the Property 'prop' with the value 'Python'")
x = A("Python")
print("The value is: ", x.prop)
print("Reassigning the Property 'prop' to 'Python descriptors'")
x.prop = "Python descriptors"
print("The value is: ", x.prop)
print(A.prop.getter(x))

def new_prop_setter(self, prop):
    if prop=="foo":
        self.__prop = "foobar"
    else:
        self.__prop = prop
A.prop.setter


__init__ of Property called with:
fget=<function A.prop at 0x10cfdf380> fset=None fdel=None doc=None

Property.setter has been called!

__init__ of Property called with:
fget=<function A.prop at 0x10cfdf380> fset=<function A.prop at 0x10cef74c0> fdel=None doc= This will be the doc string of the property 

__init__ of Property called with:
fget=<function A.prop2_getter at 0x10cef72e0> fset=<function A.prop2_setter at 0x10cfdf4c0> fdel=None doc=None
Initializing the Property 'prop' with the value 'Python'

Property.__set__ has been called!
prop will be set

Property.__get__ has been called!
The Property 'prop' will be returned now:
The value is:  Python
Reassigning the Property 'prop' to 'Python descriptors'

Property.__set__ has been called!
prop will be set

Property.__get__ has been called!
The Property 'prop' will be returned now:
The value is:  Python descriptors

Property.__get__ has been called!

Property.getter has been called!

__init__ of Property called with:
fget=<__main__.A

<bound method Property.setter of <__main__.Property object at 0x10d3e6950>>

In [12]:
class Robot:
    
    
    def __init__(self, name="Marvin", city="Freiburg"):
        self.name = name
        self.city = city
        
    @Property
    def name(self):
        return self.__name
    
    @name.setter
    def name(self, name):
        if name == "hello":
            self.__name = "hi"
        else:
            self.__name = name
            
            
x = Robot("Marvin")
print(x.name)
x.name = "Eddie"
print(x.name)


__init__ of Property called with:
fget=<function Robot.name at 0x10cfdf740> fset=None fdel=None doc=None

Property.setter has been called!

__init__ of Property called with:
fget=<function Robot.name at 0x10cfdf740> fset=<function Robot.name at 0x10cef7240> fdel=None doc=None

Property.__set__ has been called!

Property.__get__ has been called!
Marvin

Property.__set__ has been called!

Property.__get__ has been called!
Eddie


In [13]:
class A:
    def a(func):
        def helper(self, x):
            return 4 * func(self, x)
        return helper
    @a
    def b(self, x):
        return x + 1
a = A()
a.b(4)

20

De nombreuses personnes demandent s'il est possible de créer automatiquement des descripteurs au moment de l'exécution. C'est possible, comme nous le montrons dans l'exemple suivant. D'un autre côté, cet exemple n'est pas très utile, car les getters et setters n'ont aucune fonctionnalité réelle :

In [14]:
class DynPropertyClass(object):
    
    def add_property(self, attribute):
        """ add a property to the class """   
        
        def get_attribute(self):
            """ The value for attribute 'attribute' will be retrieved """
            return getattr(self, "_" + type(x).__name__ + "__" + attribute)
        
        def set_attribute(self, value):
            """ The value for attribute 'attribute' will be retrieved """
            #setter = lambda self, value: self.setProperty(attribute, value)
            setattr(self, "_" + type(x).__name__ + "__" + attribute, value)
        #construct property attribute and add it to the class
        setattr(type(self), attribute, property(fget=get_attribute, 
                                                fset=set_attribute, 
                                                doc="Auto‑generated method"))
x = DynPropertyClass()
x.add_property('name')
x.add_property('city')
x.name = "Henry"
x.name
x.city = "Hamburg"
print(x.name, x.city)
print(x.__dict__)

Henry Hamburg
{'_DynPropertyClass__name': 'Henry', '_DynPropertyClass__city': 'Hamburg'}
